In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = trial.suggest_float("subsample", 0.01, 1)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree, 'num_boost_round': num_boost_round}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/"
model_type = 'RandomForest' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 2 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = True
study_name = "study_RandomForest_20220705-12.pkl"

In [5]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

Best trial until now:
 Value:  6.710196429609631
 Params: 
    n_estimators: 150
    max_depth: 64
    max_samples: 0.8785156026362354


In [ ]:
study.optimize(objective, n_trials=250)

Split #0, This Result: 5.6260, Average Result: 5.6260
Split #1, This Result: 6.2747, Average Result: 5.9503
Split #2, This Result: 4.4762, Average Result: 5.4590
Split #3, This Result: 4.7551, Average Result: 5.2830
Split #4, This Result: 16.5594, Average Result: 7.5383
Split #5, This Result: 11.8042, Average Result: 8.2493
Split #6, This Result: 3.5641, Average Result: 7.5799
Split #7, This Result: 6.1615, Average Result: 7.4026
Split #8, This Result: 4.9851, Average Result: 7.1340


[I 2022-07-05 14:16:22,919] Trial 14 finished with value: 6.773366674795023 and parameters: {'n_estimators': 186, 'max_depth': 67, 'max_samples': 0.7089963660387368}. Best is trial 12 with value: 6.710196429609631.


Split #9, This Result: 3.5274, Average Result: 6.7734
Split #0, This Result: 5.6231, Average Result: 5.6231
Split #1, This Result: 6.2705, Average Result: 5.9468
Split #2, This Result: 4.4711, Average Result: 5.4549
Split #3, This Result: 4.3705, Average Result: 5.1838
Split #4, This Result: 16.5614, Average Result: 7.4593
Split #5, This Result: 11.7977, Average Result: 8.1824
Split #6, This Result: 3.7056, Average Result: 7.5428
Split #7, This Result: 6.1637, Average Result: 7.3705
Split #8, This Result: 5.0044, Average Result: 7.1076
Split #9, This Result: 3.5226, Average Result: 6.7491


[I 2022-07-05 14:47:58,235] Trial 15 finished with value: 6.74905605227238 and parameters: {'n_estimators': 181, 'max_depth': 95, 'max_samples': 0.9810025329922659}. Best is trial 12 with value: 6.710196429609631.


Split #0, This Result: 6.9917, Average Result: 6.9917
Split #1, This Result: 7.5253, Average Result: 7.2585
Split #2, This Result: 5.6629, Average Result: 6.7267
Split #3, This Result: 5.9972, Average Result: 6.5443
Split #4, This Result: 16.8786, Average Result: 8.6112
Split #5, This Result: 12.6662, Average Result: 9.2870
Split #6, This Result: 4.8286, Average Result: 8.6501
Split #7, This Result: 7.3104, Average Result: 8.4826
Split #8, This Result: 5.4226, Average Result: 8.1426


...

Split #9, This Result: 3.7120, Average Result: 6.8873
Split #0, This Result: 6.9587, Average Result: 6.9587
Split #1, This Result: 6.8754, Average Result: 6.9170
Split #2, This Result: 5.4490, Average Result: 6.4277
Split #3, This Result: 5.9711, Average Result: 6.3135
Split #4, This Result: 16.7492, Average Result: 8.4007
Split #5, This Result: 12.5432, Average Result: 9.0911
Split #6, This Result: 4.8172, Average Result: 8.4805
Split #7, This Result: 6.7740, Average Result: 8.2672
Split #8, This Result: 6.0271, Average Result: 8.0183


[I 2022-07-05 15:24:34,478] Trial 21 finished with value: 7.670921894205625 and parameters: {'n_estimators': 46, 'max_depth': 24, 'max_samples': 0.06854504587626338}. Best is trial 12 with value: 6.710196429609631.


Split #9, This Result: 4.5443, Average Result: 7.6709
Split #0, This Result: 5.6582, Average Result: 5.6582
Split #1, This Result: 6.2958, Average Result: 5.9770
Split #2, This Result: 4.4770, Average Result: 5.4770
Split #3, This Result: 4.9843, Average Result: 5.3539
Split #4, This Result: 16.5494, Average Result: 7.5930
Split #5, This Result: 11.8145, Average Result: 8.2966
Split #6, This Result: 3.5764, Average Result: 7.6222
Split #7, This Result: 6.1489, Average Result: 7.4381
Split #8, This Result: 5.1438, Average Result: 7.1832


[I 2022-07-05 15:27:27,097] Trial 22 finished with value: 6.822844266179125 and parameters: {'n_estimators': 16, 'max_depth': 62, 'max_samples': 0.3276088147406737}. Best is trial 12 with value: 6.710196429609631.


Split #9, This Result: 3.5800, Average Result: 6.8228


In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621